# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.


In [2]:
import numpy as np
import pandas as pd
import math
import re

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [3]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [5]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [6]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [8]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [10]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [11]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [13]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés. Nous allons translater les notes afin que la moyenne des notes pour chaque utilisateur se trouve à 0. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [258]:
movies_title = movies.loc[movies['id'].isin(ratings.movieId), ['id', 'title']]
print(len(ratings.movieId.unique()))
ratings_small = ratings.loc[ratings['userId'] <= 100]

9066


In [21]:
def mean_rating(uid):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    n = ratings.loc[ratings['userId'] == uid].count().loc['userId']
    s = ratings.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [22]:
def normalize(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées
    '''
    mean = ratings.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_rating(uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [259]:
normalize(ratings_small)
print(ratings_small.head(30))

    userId  movieId  rating  rating_norm
0        1       31     2.5    -0.050000
1        1     1029     3.0     0.450000
2        1     1061     3.0     0.450000
3        1     1129     2.0    -0.550000
4        1     1172     4.0     1.450000
5        1     1263     2.0    -0.550000
6        1     1287     2.0    -0.550000
7        1     1293     2.0    -0.550000
8        1     1339     3.5     0.950000
9        1     1343     2.0    -0.550000
10       1     1371     2.5    -0.050000
11       1     1405     1.0    -1.550000
12       1     1953     4.0     1.450000
13       1     2105     4.0     1.450000
14       1     2150     3.0     0.450000
15       1     2193     2.0    -0.550000
16       1     2294     2.0    -0.550000
17       1     2455     2.5    -0.050000
18       1     2968     1.0    -1.550000
19       1     3671     3.0     0.450000
20       2       10     4.0     0.513158
21       2       17     5.0     1.513158
22       2       39     5.0     1.513158
23       2      

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Regrouper les utilisateurs dans des peer-group

Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

In [113]:
def cor(u, v, df, Iuv):
    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [114]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [126]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [127]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

**Test des taux de corrélation sur les utilisateurs 2 et 3 qui ont 8 films en communs**

In [128]:
u, v = 2, 3
df = ratings_small
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
print(Iuv)

print(cor(u, v, df, Iuv))
# print(cor_adj(u, v, df))
print(cor_dis(u, v, df, Iuv))
# print(cor_dis_adj(u, v, df))

    movieId
27      110
49      296
57      356
64      377
79      527
88      588
91      592
92      593
-0.016060945830838957
-0.006424378332335582


Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ne stocker la corrélation pour un couple qu'une seule fois.

In [137]:
def correlation_matrix(df, cor_fct=cor):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        if not u % 20 : 
            print('user : ', u, end='')
        for j in range(i + 1, len(users)):
            v = users[j]
            Iu = df.loc[df['userId'] == u, ['movieId']]
            Iv = df.loc[df['userId'] == v, ['movieId']]
            Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [219]:
def get_cor(u, v, cm):
    '''
    Retourne le taux de correlation entre u et v stocké dans cm
    '''
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('-inf')

In [260]:
cm_small = correlation_matrix(ratings_small)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


user :  20user :  40user :  60user :  80user :  100

In [261]:
print(cm_small.head(10))
print(cm_small.loc[(1, 9)])
print(get_cor(1, 9, cm_small))

      correlation
u v              
1 4      0.042137
  5     -1.000000
  7     -0.752427
  9      1.000000
  15     0.043773
  17    -0.972281
  19     0.053458
  20    -1.000000
  21    -0.478805
  22     0.701197
correlation    1.0
Name: (1, 9), dtype: float64
1.0


Nous allons maintenant prédire la note qu'un utilisateur **u** donnerait à un film *m . Pour cela, nous allons faire la somme des notes données à l'item *i* par les k utilisateurs plus proches de u qui ont donné une note à m. Cette somme sera pondérée par les coéfficients de corrélations.

$$
\hat{\sigma_{um}} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

Le peer-group de l'utilisateur u pour le film m est l'ensemble des k utilisateurs qui ont donné une note au film m les plus proche de l'utilisateur u en terme de taux de corrélation. 

In [288]:
def peers(user, movie, k, cm, df):
    '''
    Retourne les k utilisateurs du peer-group de (user, movie)
    '''
    top = [(float('-inf'), user)] * k
    df_movie = df.loc[df['movieId'] == movie, ['userId', 'rating_norm']]
    for v in df_movie.userId.unique():
        taux = get_cor(user, v, cm)
        if taux and taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [265]:
print(peers(1, 10, 4, cm_small, ratings_small))

[39, 19, 15, 4]


In [286]:
def predict(user, movie, cm, df):
    res = mean_rating(user)
    peer_group = peers(user, movie, 4, cm, df)
    for v in peer_group:
        svm = float(df.loc[(df['userId'] == v) & (df['movieId'] == movie), 'rating_norm'])
        res += svm * get_cor(user, v, cm) / abs(get_cor(user, v, cm))
    return res

In [291]:
friends = peers(1, 10, 4, cm_small, ratings_small)
p = predict(1, 10, cm_small, ratings_small)
print(friends)
print(p)
print(df.loc[(df['userId'].isin(friends)) & (df['movieId'] == 10)])

[39, 19, 15, 4]
3.2555945379753366
      userId  movieId  rating  rating_norm
147        4       10     4.0    -0.348039
966       15       10     3.0     0.378235
3112      19       10     3.0    -0.534279


3.2555945379753366
